In [2]:
import re
import pandas as pd
from textblob import TextBlob

In [3]:
def isNaN(num):
    return num != num

In [6]:
df = pd.read_csv('humans.csv', sep=';', index_col='id', 
                 usecols = ['id', 'source', 'is_retweet', 'hashtags', 'lemma_filter'])#.sample(frac=0.1)

In [7]:
df0 = df[df['is_retweet'] == False].sample(frac=0.1)
df0_1 = df0[df0.apply(lambda x: (isNaN(x['lemma_filter'])==False) and (isNaN(x['hashtags'])==False), axis=1)]
df0_1['hashtags_list'] = df0_1.apply(lambda x: [i for i in set(re.findall(r"[\w']+", x['hashtags'].lower())) if len(i) > 1], axis=1)
df1 = df0_1[df0_1.apply(lambda x: len(x['hashtags_list']) > 0, axis=1)]

/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df1.head(5)

,source,is_retweet,hashtags,lemma_filter,hashtags_list
id,,,,,
224766,instagram,False,"#stopthesteal, #demorats, #donaldtrump, #metoo...",gay billionaire & co- founder paypal might cal...,"[murica, trump2020, merica, maga, makeamericag..."
336223,instagram,False,"#Moon, #Gemini, #Libra,, #Aquarius., #connecti...",today ( bihday ! 😊 ) around 7am mst ( subconsc...,"[lilith, goddess, partnerships, queen, juno, r..."
39406,twitter,False,metoo,side note : layer film . movie abt female empo...,[metoo]
204346,instagram,False,"#transgender, #mtf, #lgbtqia, #trans, #transge...","sit , morning , diner , corner , wait , counte...","[pansexual, thisiswhattranslookslike, eyeliner..."
16646,instagram,False,"#kinderzimmer, #metoo, #momsofinstagram, #chil...","rehpuppen , einfach zuckerl . .erhältlich unse...","[momlife, mamablogger_de, kidsroom, lebenmitki..."


In [9]:
df2 = df1
df2['sentiment_polarity'] = df2.apply(lambda x: TextBlob(x['lemma_filter']).sentiment[0], axis=1)
df2['sentiment'] = df2.apply(lambda x: 'pos' if x['sentiment_polarity'] > 0 else 'neg' if x['sentiment_polarity'] < 0 else 'neutral', axis=1)



/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df2.head(5)

,source,is_retweet,hashtags,lemma_filter,hashtags_list,sentiment_polarity,sentiment
id,,,,,,,
224766,instagram,False,"#stopthesteal, #demorats, #donaldtrump, #metoo...",gay billionaire & co- founder paypal might cal...,"[murica, trump2020, merica, maga, makeamericag...",0.208333,pos
336223,instagram,False,"#Moon, #Gemini, #Libra,, #Aquarius., #connecti...",today ( bihday ! 😊 ) around 7am mst ( subconsc...,"[lilith, goddess, partnerships, queen, juno, r...",0.221429,pos
39406,twitter,False,metoo,side note : layer film . movie abt female empo...,[metoo],0.000000,neutral
204346,instagram,False,"#transgender, #mtf, #lgbtqia, #trans, #transge...","sit , morning , diner , corner , wait , counte...","[pansexual, thisiswhattranslookslike, eyeliner...",0.000000,neutral
16646,instagram,False,"#kinderzimmer, #metoo, #momsofinstagram, #chil...","rehpuppen , einfach zuckerl . .erhältlich unse...","[momlife, mamablogger_de, kidsroom, lebenmitki...",0.000000,neutral


In [17]:
NUM = 50  #UM of most popular hashtags
# # Only selecting positive and negative sentiment
# # df3 = df2[df2['sentiment'] != 'neutral'].set_index(['sentiment'])['hashtags_list'].apply(pd.Series).stack()

# Select all three sentiments
df3 = df2.set_index(['sentiment'])['hashtags_list'].apply(pd.Series).stack()
df3 = df3.reset_index()

df3.columns = ['sentiment', 'sample_num','hashtag']
df3_1 = df3.groupby(['hashtag', 'sentiment']).size().reset_index(name='counts')
df3_2 = df3.groupby(['hashtag']).size().reset_index(name='counts')
df3_merged = pd.merge(df3_1, df3_2, on='hashtag', how='inner')

result = df3_merged[df3_merged['counts_y'] > NUM] #df3_merged.nlargest(NUM, 'counts_y') 
result['probe'] = result.apply(lambda x: float(x['counts_x'])/x['counts_y'], axis=1)
# result = result[result.apply(lambda x: 'metoo' not in x['hashtag'], axis=1)]

hashtag_dic = {}
for index, row in result.iterrows():
    hashtag_dic[row['sentiment'], row['hashtag']] = row['probe']

hashtag_dic

# 2019womensmarch

#
hashtags_set = set(result['hashtag'].values)
len(hashtags_set)

/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


1074

In [14]:
df1['idx'] = df1.index
tdf = df1.set_index(['idx'])['hashtags_list'].apply(pd.Series).stack()
tdf = tdf.reset_index()
tdf.columns = ['idx', 'sample_num','hashtag']
ht_dic = {}
for index, row in tdf.iterrows():
    if row['hashtag'] in ht_dic:
        ht_dic[row['hashtag']].add(row['idx'])
    else:
        ht_dic[row['hashtag']] = set([row['idx']])

ht_dic

/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'murica': {143711, 223071, 224766, 242697, 275475, 307390, 309978, 321383},
 'trump2020': {1525,
  4386,
  4407,
  6313,
  6345,
  6352,
  6354,
  6912,
  6913,
  6949,
  6954,
  6964,
  8166,
  8391,
  9745,
  18593,
  19599,
  22486,
  25526,
  25538,
  25655,
  28286,
  28287,
  28295,
  31074,
  31497,
  32351,
  32773,
  33298,
  33299,
  33465,
  33701,
  34420,
  34438,
  35175,
  35243,
  35563,
  36251,
  36313,
  36326,
  36379,
  36430,
  36431,
  36435,
  36513,
  36604,
  36847,
  36852,
  36854,
  36893,
  37323,
  37387,
  37936,
  38346,
  39274,
  39497,
  40119,
  40123,
  40184,
  40625,
  40849,
  40894,
  41178,
  41182,
  41305,
  41775,
  41945,
  42009,
  42056,
  42093,
  42618,
  50334,
  61868,
  64438,
  70508,
  79928,
  81155,
  83631,
  84068,
  84124,
  97481,
  107243,
  114973,
  117854,
  122644,
  127929,
  131747,
  138264,
  141101,
  142680,
  142747,
  143747,
  148206,
  148890,
  177693,
  178187,
  179718,
  183625,
  191765,
  193707,
  2051

In [19]:
import itertools
SI = {}
for (i, j) in itertools.combinations(list(hashtags_set), 2):
    tmp = 0
    if i in ht_dic:
        i_cnt = len(ht_dic[i])
        tmp += 1
    if j in ht_dic:
        j_cnt = len(ht_dic[j])
        tmp += 1
    if tmp == 2:
        i_j_cnt = len(ht_dic[i].intersection(ht_dic[j]))
    if i_j_cnt > 0:
        SI[i, j] = float(i_j_cnt)/(i_cnt + j_cnt)

In [20]:
SI

{('relatable', 'design'): 0.0014005602240896359,
 ('relatable', 'maga'): 0.0013812154696132596,
 ('relatable', 'spicymemes'): 0.006289308176100629,
 ('relatable', 'artwork'): 0.002336448598130841,
 ('relatable', 'instagood'): 0.004153686396677051,
 ('relatable', 'cringe'): 0.020356234096692113,
 ('relatable', 'follow4followback'): 0.003125,
 ('relatable', 'success'): 0.0012239902080783353,
 ('relatable', 'cute'): 0.008,
 ('relatable', 'qualitymemes'): 0.3,
 ('relatable', 'business'): 0.0019157088122605363,
 ('relatable', 'bollywood'): 0.0041928721174004195,
 ('relatable', 'writersofinstagram'): 0.003937007874015748,
 ('relatable', 'lol'): 0.07692307692307693,
 ('relatable', 'wtf'): 0.0033333333333333335,
 ('relatable', 'metoo'): 0.008937401560382403,
 ('relatable', 'shopping'): 0.0030211480362537764,
 ('relatable', 'dance'): 0.002652519893899204,
 ('relatable', 'sjw'): 0.016483516483516484,
 ('relatable', 'loganpaul'): 0.009584664536741214,
 ('relatable', 'share'): 0.004366812227074236

In [12]:
import pickle
with open('hashtag.pickle', 'wb') as handle:
    pickle.dump([list(hashtags_set), hashtag_dic, SI], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
hashtag_dic

{('neg', '16days'): 0.11940298507462686,
 ('neutral', '16days'): 0.3283582089552239,
 ('pos', '16days'): 0.5522388059701493,
 ('neg', '16daysofactivism'): 0.16129032258064516,
 ('neutral', '16daysofactivism'): 0.22580645161290322,
 ('pos', '16daysofactivism'): 0.6129032258064516,
 ('neg', '2018'): 0.047058823529411764,
 ('neutral', '2018'): 0.6176470588235294,
 ('pos', '2018'): 0.3352941176470588,
 ('neg', '2019'): 0.09507042253521127,
 ('neutral', '2019'): 0.4894366197183099,
 ('pos', '2019'): 0.4154929577464789,
 ('neg', '420'): 0.07317073170731707,
 ('neutral', '420'): 0.7195121951219512,
 ('pos', '420'): 0.2073170731707317,
 ('neutral', 'abc'): 0.07272727272727272,
 ('pos', 'abc'): 0.9272727272727272,
 ('pos', 'abhijitnaskar'): 1.0,
 ('neg', 'abortion'): 0.17424242424242425,
 ('neutral', 'abortion'): 0.553030303030303,
 ('pos', 'abortion'): 0.2727272727272727,
 ('neg', 'abstract'): 0.07017543859649122,
 ('neutral', 'abstract'): 0.45614035087719296,
 ('pos', 'abstract'): 0.473684210